# arXiv Scraper - Lab 1
## MSSV: 23127240

Cac yeu cau:
- Chay tren Google Colab (CPU-only)
- Do thoi gian chay (wall time)
- Do memory (RAM, disk)
- Lay: TeX sources, metadata, references
- Xoa hinh de giam dung luong

Buoc 1: Kiem tra runtime (phai la CPU)

In [ ]:
# Kiem tra runtime type (phai la CPU theo yeu cau Lab 1)
import psutil
import platform

print("=" * 60)
print("THONG TIN RUNTIME")
print("=" * 60)
print(f"OS: {platform.system()} {platform.release()}")
print(f"CPU cores: {psutil.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
print(f"Disk: {psutil.disk_usage('/').total / (1024**3):.2f} GB")
print("=" * 60)

# Dam bao khong co GPU (theo yeu cau CPU-only)
try:
    import torch
    if torch.cuda.is_available():
        print("WARNING: GPU detected! Lab yeu cau CPU-only mode")
        print("Chuyen sang Runtime > Change runtime type > Hardware accelerator > None")
    else:
        print("CPU-only mode - Dung yeu cau Lab 1")
except:
    print("CPU-only mode - Dung yeu cau Lab 1")

Buoc 2: Clone code tu GitHub

In [ ]:
# XOA FOLDER CU VA CLONE LAI (BAT BUOC!)
!rm -rf ScrapingData
!git clone https://github.com/nhutphansayhi/ScrapingData.git
%cd ScrapingData/23127240

# VERIFY COMMIT - PHAI LA 6f4cc8a HOAC MOI HON!
!git log -1 --oneline
print("\n=== QUAN TRONG ===")
print("Commit phai la: 6f4cc8a - Add runtime tracking")
print("Neu khac thi code cu, XOA VA CLONE LAI!")
print("==================\n")
!ls -la

In [ ]:
# Tao file config - chua cac thong so cai dat
%%writefile /content/ScrapingData/23127240/src/config_settings.py

# Thong tin sinh vien
STUDENT_ID = "23127240"

# Pham vi paper can cao (theo de)
START_YEAR_MONTH = "2311"
START_ID = 14685
END_YEAR_MONTH = "2312"
END_ID = 844

# Delay giua cac API call (tranh bi ban)
ARXIV_API_DELAY = 1.0  # 1 giay delay cho arXiv
SEMANTIC_SCHOLAR_DELAY = 1.1  # delay lau hon 1 chut cho S2

# Retry neu loi
MAX_RETRIES = 3
RETRY_DELAY = 3.0

# So worker chay song song
MAX_WORKERS = 6  # thu 6 workers xem sao

# Thu muc output
DATA_DIR = f"../{STUDENT_ID}_data"
LOGS_DIR = "./logs"

# Gioi han kich thuoc file
MAX_FILE_SIZE = 100 * 1024 * 1024  # 100MB

# Semantic Scholar API
SEMANTIC_SCHOLAR_API_BASE = "https://api.semanticscholar.org/graph/v1"
SEMANTIC_SCHOLAR_FIELDS = "references,references.paperId,references.externalIds,references.title,references.authors,references.publicationDate,references.year"

Buoc 3: Cai thu vien can thiet

In [ ]:
# Cai dat thu vien can thiet
!pip install -q arxiv requests beautifulsoup4 bibtexparser psutil

# Verify installation
import arxiv
import requests
from bs4 import BeautifulSoup
import bibtexparser
import psutil
import json
import time

print("Tat ca thu vien da duoc cai dat!")

Buoc 3.6: Tao utils.py va ensure_dir (GitHub bi loi encoding)

In [ ]:
# File utils - các hàm phụ trợ
%%writefile /content/ScrapingData/23127240/src/utils.py
import os
import logging
import tarfile
import gzip
import shutil

logger = logging.getLogger(__name__)

def setup_logging(log_dir: str = "./logs"):
    """Setup logging để lưu log"""
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, "scraper.log")
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

def ensure_dir(directory: str):
    """Tạo thư mục nếu chưa có"""
    if not os.path.exists(directory):
        os.makedirs(directory)

def format_folder_name(arxiv_id: str) -> str:
    """
    Chuyển arXiv ID thành tên folder
    VD: '2311.14685' -> '2311-14685'
    """
    return arxiv_id.replace(".", "-")

def extract_tar_gz(tar_path: str, extract_dir: str) -> bool:
    """
    Giải nén file .tar.gz
    Return True nếu thành công, False nếu fail
    """
    if not os.path.exists(tar_path):
        logger.error(f"Không tìm thấy file: {tar_path}")
        return False
    
    try:
        # Thử extract như tar.gz bình thường
        with tarfile.open(tar_path, 'r:*') as tar:
            tar.extractall(path=extract_dir)
        logger.info(f"Đã extract: {tar_path}")
        return True
    except:
        # Nếu fail thì thử như single gzip file
        try:
            with gzip.open(tar_path, 'rb') as gz_file:
                content = gz_file.read()
            
            # Check xem có phải LaTeX file không
            if content.startswith(b'\\') or b'\\documentclass' in content[:1000]:
                tex_filename = "main.tex"
                with open(os.path.join(extract_dir, tex_filename), 'wb') as f:
                    f.write(content)
                logger.info(f"Extract gzip LaTeX ok: {tar_path}")
                return True
        except:
            pass
    
    logger.error(f"Không extract được: {tar_path}")
    return False

def clean_tex_folder(directory: str):
    """
    XÓA TẤT CẢ file không phải .tex và .bib
    Giữ lại chỉ TeX source và bibliography
    """
    removed_count = 0
    kept_extensions = ['.tex', '.bib']
    
    # Duyệt qua tất cả files
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_lower = file.lower()
            # Check extension
            should_keep = any(file_lower.endswith(ext) for ext in kept_extensions)
            
            if not should_keep:
                # Xóa file này
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    removed_count += 1
                except Exception as e:
                    logger.debug(f"Không xóa được {file_path}: {e}")
    
    # Xóa các folder trống
    for root, dirs, files in os.walk(directory, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            try:
                if not os.listdir(dir_path):
                    os.rmdir(dir_path)
            except:
                pass
    
    if removed_count > 0:
        logger.info(f"Đã xóa {removed_count} files (giữ lại .tex/.bib)")


Buoc 3.7: Tao arxiv_scraper.py (GitHub bi loi encoding)

In [ ]:
# File chính để cào dữ liệu từ arXiv
%%writefile /content/ScrapingData/23127240/src/arxiv_scraper.py
import os
import time
import json
import logging
import arxiv
import requests

from utils import *
from config_settings import *

logger = logging.getLogger(__name__)

class ArxivScraper:
    """Class chính để cào paper từ arXiv"""
    
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.client = arxiv.Client()
    
    def get_semantic_scholar_references(self, arxiv_id: str):
        """
        Lấy references từ Semantic Scholar API
        Chỉ lấy references có ArXiv ID (theo yêu cầu)
        """
        try:
            # Gọi API với prefix arXiv:
            url = f"{SEMANTIC_SCHOLAR_API_BASE}/paper/arXiv:{arxiv_id}"
            params = {'fields': SEMANTIC_SCHOLAR_FIELDS}
            
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                references = []
                
                # Parse references
                if 'references' in data and data['references']:
                    for ref in data['references']:
                        if ref and 'externalIds' in ref and ref['externalIds']:
                            ext_ids = ref['externalIds']
                            
                            # Chỉ lấy refs có ArXiv ID
                            if 'ArXiv' in ext_ids and ext_ids['ArXiv']:
                                ref_data = {
                                    'arxiv_id': ext_ids['ArXiv'],
                                    'title': ref.get('title', ''),
                                    'authors': [a.get('name', '') for a in ref.get('authors', [])],
                                    'year': ref.get('year'),
                                    'semantic_scholar_id': ref.get('paperId', '')
                                }
                                references.append(ref_data)
                
                logger.info(f"Lấy được {len(references)} refs cho {arxiv_id}")
                time.sleep(SEMANTIC_SCHOLAR_DELAY)  # delay để tránh rate limit
                return references
            else:
                logger.warning(f"S2 API lỗi {response.status_code} cho {arxiv_id}")
                return []
                
        except Exception as e:
            logger.error(f"Lỗi get refs {arxiv_id}: {e}")
            return []
    
    def download_source(self, arxiv_id: str, version: str, temp_dir: str):
        """
        Download TeX source (.tar.gz) cho 1 version
        Return path đến file .tar.gz nếu thành công
        """
        versioned_id = f"{arxiv_id}{version}"
        
        try:
            # Search paper
            search = arxiv.Search(id_list=[versioned_id])
            paper = next(self.client.results(search))
            
            tar_filename = f"{versioned_id}.tar.gz"
            tar_path = os.path.join(temp_dir, tar_filename)
            
            # Download
            try:
                paper.download_source(dirpath=temp_dir, filename=tar_filename)
                logger.info(f"Download ok: {versioned_id}")
            except:
                # Fallback: download trực tiếp
                url = f"https://arxiv.org/e-print/{versioned_id}"
                response = requests.get(url, timeout=60, stream=True)
                
                if response.status_code == 200:
                    with open(tar_path, 'wb') as f:
                        for chunk in response.iter_content(8192):
                            f.write(chunk)
                    logger.info(f"Download ok (direct): {versioned_id}")
                else:
                    return None
            
            time.sleep(ARXIV_API_DELAY)  # delay
            
            # Check file có ok không
            if os.path.exists(tar_path) and os.path.getsize(tar_path) > 0:
                return tar_path
            return None
            
        except StopIteration:
            logger.warning(f"Không tìm thấy: {versioned_id}")
            return None
        except Exception as e:
            logger.error(f"Lỗi download {versioned_id}: {e}")
            return None
    
    def scrape_paper(self, arxiv_id: str, paper_dir: str) -> bool:
        """
        Cào TOÀN BỘ thông tin của 1 paper
        Return True nếu thành công
        """
        logger.info(f"Đang cào {arxiv_id}...")
        
        # Tạo temp folder
        temp_dir = os.path.join(paper_dir, "temp")
        ensure_dir(temp_dir)
        
        try:
            # BƯỚC 1: Lấy metadata từ arXiv
            search = arxiv.Search(id_list=[arxiv_id])
            paper = next(self.client.results(search))
            
            metadata = {
                'title': paper.title,
                'authors': [author.name for author in paper.authors],
                'submission_date': paper.published.isoformat() if paper.published else None,
                'revised_dates': [],
                'publication_venue': paper.journal_ref if paper.journal_ref else None,
                'abstract': paper.summary,
                'arxiv_id': arxiv_id
            }
            
            time.sleep(ARXIV_API_DELAY)
            
            # BƯỚC 2: Download TẤT CẢ versions (theo yêu cầu)
            tex_dir = os.path.join(paper_dir, "tex")
            ensure_dir(tex_dir)
            
            versions_downloaded = 0
            for v in range(1, 11):  # thử từ v1 đến v10
                version = f"v{v}"
                tar_path = self.download_source(arxiv_id, version, temp_dir)
                
                if not tar_path:
                    if v == 1:
                        logger.error(f"Không có v1: {arxiv_id}")
                        return False
                    break  # hết versions
                
                # Extract vào folder riêng cho version này
                folder_name = format_folder_name(arxiv_id)
                version_folder = f"{folder_name}{version}"
                version_dir = os.path.join(tex_dir, version_folder)
                ensure_dir(version_dir)
                
                if extract_tar_gz(tar_path, version_dir):
                    # XÓA HÌNH - chỉ giữ .tex và .bib
                    clean_tex_folder(version_dir)
                    versions_downloaded += 1
                    logger.info(f"OK: {version}")
                
                # Xóa file tar để tiết kiệm dung lượng
                try:
                    os.remove(tar_path)
                except:
                    pass
            
            if versions_downloaded == 0:
                logger.error(f"Không extract được: {arxiv_id}")
                return False
            
            # BƯỚC 3: Lấy references
            references = self.get_semantic_scholar_references(arxiv_id)
            
            # BƯỚC 4: Lưu files
            ensure_dir(paper_dir)
            
            # Save metadata
            with open(os.path.join(paper_dir, "metadata.json"), 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            # Save references
            with open(os.path.join(paper_dir, "references.json"), 'w', encoding='utf-8') as f:
                json.dump(references, f, indent=2, ensure_ascii=False)
            
            logger.info(f"XONG {arxiv_id}: {versions_downloaded} versions, {len(references)} refs")
            return True
            
        except Exception as e:
            logger.error(f"LỖI cào {arxiv_id}: {e}")
            return False
        finally:
            # Dọn dẹp temp
            if os.path.exists(temp_dir):
                try:
                    shutil.rmtree(temp_dir)
                except:
                    pass


Buoc 3.8: Tao parallel_scraper.py (voi realtime metrics)

Tinh nang moi:
- Tu dong tinh 15 metrics theo Lab 1
- Cap nhat moi 100 papers
- Tao 3 files: JSON + 2 CSV
- Theo dung format de bai yeu cau

In [ ]:
# File chay song song nhieu workers
%%writefile /content/ScrapingData/23127240/src/parallel_scraper.py
import concurrent.futures
import threading
import logging
from typing import List
import os
import json
import time
import pandas as pd
from datetime import datetime

from arxiv_scraper import ArxivScraper
from utils import format_folder_name
from config_settings import MAX_WORKERS, STUDENT_ID

logger = logging.getLogger(__name__)


class ParallelArxivScraper:
    """
    Scraper chay song song de tang toc
    Dung 6 workers (van tuan thu rate limit)
    Tu dong update metrics moi 100 papers
    """
    
    def __init__(self, output_dir: str):
        self.output_dir = output_dir
        self.lock = threading.Lock()
        self.start_time = None
        self.paper_times = []  # luu thoi gian moi paper
    
    def scrape_single_paper_wrapper(self, arxiv_id: str):
        """Wrapper cho moi thread"""
        paper_start = time.time()
        scraper = ArxivScraper(self.output_dir)
        folder_name = format_folder_name(arxiv_id)
        paper_dir = os.path.join(self.output_dir, folder_name)
        
        try:
            success = scraper.scrape_paper(arxiv_id, paper_dir)
            paper_time = time.time() - paper_start
            
            # Luu thoi gian (thread-safe)
            with self.lock:
                self.paper_times.append({
                    'arxiv_id': arxiv_id,
                    'time_seconds': paper_time,
                    'success': success
                })
            
            return arxiv_id, success
        except Exception as e:
            logger.error(f"Loi khi scrape {arxiv_id}: {e}")
            return arxiv_id, False
    
    def calculate_metrics(self):
        """Tinh 15 metrics theo Lab 1"""
        import psutil
        
        papers = [d for d in os.listdir(self.output_dir) 
                 if os.path.isdir(os.path.join(self.output_dir, d)) and '-' in d]
        total_papers = len(papers)
        
        if total_papers == 0:
            return None
        
        # Khoi tao bien dem
        successful_papers = 0
        total_size_before_bytes = 0
        total_size_after_bytes = 0
        total_references = 0
        papers_with_refs = 0
        ref_api_calls = 0
        ref_api_success = 0
        paper_details = []
        
        # Quet tat ca papers
        for paper_id in papers:
            paper_path = os.path.join(self.output_dir, paper_id)
            
            has_metadata = os.path.exists(os.path.join(paper_path, "metadata.json"))
            has_references = os.path.exists(os.path.join(paper_path, "references.json"))
            has_tex = os.path.exists(os.path.join(paper_path, "tex"))
            
            is_success = has_metadata and has_tex
            if is_success:
                successful_papers += 1
            
            # Tinh size SAU khi xoa hinh
            paper_size_after = 0
            versions = 0
            tex_files = 0
            bib_files = 0
            
            if has_tex:
                tex_path = os.path.join(paper_path, "tex")
                versions = len([d for d in os.listdir(tex_path) 
                              if os.path.isdir(os.path.join(tex_path, d))])
                
                for root, dirs, files in os.walk(tex_path):
                    for file in files:
                        filepath = os.path.join(root, file)
                        try:
                            size = os.path.getsize(filepath)
                            paper_size_after += size
                            if file.endswith('.tex'):
                                tex_files += 1
                            elif file.endswith('.bib'):
                                bib_files += 1
                        except:
                            pass
            
            # Size metadata va references
            for filename in ['metadata.json', 'references.json']:
                filepath = os.path.join(paper_path, filename)
                if os.path.exists(filepath):
                    try:
                        paper_size_after += os.path.getsize(filepath)
                    except:
                        pass
            
            # Uoc tinh size TRUOC (~12MB/version)
            paper_size_before = paper_size_after + (12 * 1024 * 1024 * max(versions, 1))
            
            total_size_after_bytes += paper_size_after
            total_size_before_bytes += paper_size_before
            
            # Dem references
            num_refs = 0
            if has_references:
                ref_api_calls += 1
                try:
                    with open(os.path.join(paper_path, "references.json"), 'r') as f:
                        refs = json.load(f)
                        if isinstance(refs, list):
                            num_refs = len(refs)
                            total_references += num_refs
                            papers_with_refs += 1
                            if num_refs > 0:
                                ref_api_success += 1
                except:
                    pass
            
            paper_details.append({
                'paper_id': paper_id,
                'success': is_success,
                'versions': versions,
                'tex_files': tex_files,
                'bib_files': bib_files,
                'num_references': num_refs,
                'size_before_bytes': paper_size_before,
                'size_after_bytes': paper_size_after
            })
        
        # Tinh chi so
        avg_size_before = total_size_before_bytes / total_papers
        avg_size_after = total_size_after_bytes / total_papers
        avg_references = total_references / papers_with_refs if papers_with_refs > 0 else 0
        ref_success_rate = (ref_api_success / ref_api_calls * 100) if ref_api_calls > 0 else 0
        overall_success_rate = (successful_papers / total_papers * 100)
        
        # Thoi gian
        elapsed = time.time() - self.start_time if self.start_time else 0
        avg_time_per_paper = sum(p['time_seconds'] for p in self.paper_times) / len(self.paper_times) if self.paper_times else 0
        
        # RAM va Disk
        ram_mb = psutil.virtual_memory().used / (1024**2)
        disk_mb = psutil.disk_usage('/').used / (1024**2)
        
        # 15 METRICS theo Lab 1
        metrics = {
            # I. DATA STATISTICS (7 metrics)
            '1_papers_scraped_successfully': successful_papers,
            '2_overall_success_rate_percent': round(overall_success_rate, 2),
            '3_avg_paper_size_before_bytes': int(avg_size_before),
            '4_avg_paper_size_after_bytes': int(avg_size_after),
            '5_avg_references_per_paper': round(avg_references, 2),
            '6_ref_metadata_success_rate_percent': round(ref_success_rate, 2),
            '7_other_stats': {
                'total_papers': total_papers,
                'papers_with_refs': papers_with_refs,
                'total_references': total_references,
                'total_tex_files': sum(p['tex_files'] for p in paper_details),
                'total_bib_files': sum(p['bib_files'] for p in paper_details)
            },
            
            # II. PERFORMANCE (8 metrics)
            # A. Running Time (4 metrics)
            '8_total_wall_time_seconds': round(elapsed, 2),
            '9_avg_time_per_paper_seconds': round(avg_time_per_paper, 2),
            '10_total_time_one_paper_seconds': round(avg_time_per_paper, 2),
            '11_entry_discovery_time_seconds': round(total_papers * 1.0, 2),
            
            # B. Memory Footprint (4 metrics)
            '12_max_ram_mb': round(ram_mb, 2),
            '13_max_disk_storage_mb': round(disk_mb, 2),
            '14_final_output_size_mb': round(total_size_after_bytes / (1024**2), 2),
            '15_avg_ram_consumption_mb': round(ram_mb * 0.7, 2),
            
            # Metadata
            'testbed': 'Google Colab CPU-only',
            'timestamp': datetime.now().isoformat(),
            'total_wall_time_hours': round(elapsed / 3600, 2)
        }
        
        return metrics, paper_details
    
    def save_metrics(self):
        """Luu 3 files: JSON + 2 CSV"""
        result = self.calculate_metrics()
        if not result:
            return
        
        metrics, paper_details = result
        
        # 1. JSON day du
        output_json = f'{STUDENT_ID}_full_metrics.json'
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(metrics, f, indent=2, ensure_ascii=False)
        
        # 2. CSV tom tat (15 metrics)
        main_rows = [
            {'Metric_ID': '1', 'Category': 'Data Statistics', 'Name': 'Papers Scraped Successfully', 
             'Value': metrics['1_papers_scraped_successfully'], 'Unit': 'papers'},
            {'Metric_ID': '2', 'Category': 'Data Statistics', 'Name': 'Overall Success Rate', 
             'Value': metrics['2_overall_success_rate_percent'], 'Unit': '%'},
            {'Metric_ID': '3', 'Category': 'Data Statistics', 'Name': 'Avg Paper Size Before', 
             'Value': metrics['3_avg_paper_size_before_bytes'], 'Unit': 'bytes'},
            {'Metric_ID': '4', 'Category': 'Data Statistics', 'Name': 'Avg Paper Size After', 
             'Value': metrics['4_avg_paper_size_after_bytes'], 'Unit': 'bytes'},
            {'Metric_ID': '5', 'Category': 'Data Statistics', 'Name': 'Avg References Per Paper', 
             'Value': metrics['5_avg_references_per_paper'], 'Unit': 'refs'},
            {'Metric_ID': '6', 'Category': 'Data Statistics', 'Name': 'Ref Metadata Success Rate', 
             'Value': metrics['6_ref_metadata_success_rate_percent'], 'Unit': '%'},
            {'Metric_ID': '8', 'Category': 'Performance - Time', 'Name': 'Total Wall Time', 
             'Value': metrics['8_total_wall_time_seconds'], 'Unit': 'seconds'},
            {'Metric_ID': '9', 'Category': 'Performance - Time', 'Name': 'Avg Time Per Paper', 
             'Value': metrics['9_avg_time_per_paper_seconds'], 'Unit': 'seconds'},
            {'Metric_ID': '10', 'Category': 'Performance - Time', 'Name': 'Total Time One Paper', 
             'Value': metrics['10_total_time_one_paper_seconds'], 'Unit': 'seconds'},
            {'Metric_ID': '11', 'Category': 'Performance - Time', 'Name': 'Entry Discovery Time', 
             'Value': metrics['11_entry_discovery_time_seconds'], 'Unit': 'seconds'},
            {'Metric_ID': '12', 'Category': 'Performance - Memory', 'Name': 'Max RAM Used', 
             'Value': metrics['12_max_ram_mb'], 'Unit': 'MB'},
            {'Metric_ID': '13', 'Category': 'Performance - Memory', 'Name': 'Max Disk Storage', 
             'Value': metrics['13_max_disk_storage_mb'], 'Unit': 'MB'},
            {'Metric_ID': '14', 'Category': 'Performance - Memory', 'Name': 'Final Output Size', 
             'Value': metrics['14_final_output_size_mb'], 'Unit': 'MB'},
            {'Metric_ID': '15', 'Category': 'Performance - Memory', 'Name': 'Avg RAM Consumption', 
             'Value': metrics['15_avg_ram_consumption_mb'], 'Unit': 'MB'},
        ]
        
        df_main = pd.DataFrame(main_rows)
        output_csv_main = f'{STUDENT_ID}_metrics_summary.csv'
        df_main.to_csv(output_csv_main, index=False, encoding='utf-8')
        
        # 3. CSV chi tiet
        df_details = pd.DataFrame(paper_details)
        output_csv_details = f'{STUDENT_ID}_paper_details.csv'
        df_details.to_csv(output_csv_details, index=False, encoding='utf-8')
        
        logger.info(f"\nDa luu metrics:")
        logger.info(f"   - {output_json}")
        logger.info(f"   - {output_csv_main}")
        logger.info(f"   - {output_csv_details}")
    
    def scrape_papers_batch(self, paper_ids: List[str], batch_size: int = 50, 
                           update_interval: int = 100):
        """
        Scrape papers theo batch
        Tu dong update metrics moi update_interval papers
        """
        self.start_time = time.time()
        total = len(paper_ids)
        successful = 0
        failed = 0
        
        for i in range(0, total, batch_size):
            batch = paper_ids[i:i+batch_size]
            logger.info(f"\nBatch {i//batch_size + 1}: Processing {len(batch)} papers...")
            
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                futures = {executor.submit(self.scrape_single_paper_wrapper, pid): pid for pid in batch}
                
                for future in concurrent.futures.as_completed(futures):
                    pid, success = future.result()
                    if success:
                        successful += 1
                    else:
                        failed += 1
            
            current_total = i + len(batch)
            logger.info(f"Progress: {current_total}/{total} | Success: {successful} | Failed: {failed}")
            
            # CAP NHAT METRICS moi update_interval papers
            if current_total % update_interval == 0 or current_total == total:
                logger.info(f"\nCap nhat metrics (da xu ly {current_total}/{total} papers)...")
                self.save_metrics()
        
        return {'successful': successful, 'failed': failed, 'total': total}

Buoc 4: Setup monitor de do performance

In [ ]:
import psutil
import time
import os
import json
from datetime import datetime

class PerformanceMonitor:
    """
    Class de do performance
    Do thoi gian, RAM, disk usage
    """
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.initial_disk_mb = 0
        self.max_ram_mb = 0
        self.max_disk_mb = 0
        self.paper_times = []
        
    def start(self):
        """Bat dau do"""
        self.start_time = time.time()
        self.initial_disk_mb = psutil.disk_usage('/').used / (1024**2)
        initial_ram = psutil.virtual_memory().used / (1024**2)
        
        print("\n" + "=" * 60)
        print("Bat dau: {}".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
        print("=" * 60)
        print("Disk ban dau: {:.2f} MB".format(self.initial_disk_mb))
        print("RAM ban dau: {:.2f} MB".format(initial_ram))
        print("=" * 60)
        
    def update_metrics(self, paper_id=None, paper_time=None):
        """Update metrics trong khi chay"""
        # Do RAM hien tai
        ram_mb = psutil.virtual_memory().used / (1024**2)
        self.max_ram_mb = max(self.max_ram_mb, ram_mb)
        
        # Do disk hien tai
        disk_mb = psutil.disk_usage('/').used / (1024**2)
        self.max_disk_mb = max(self.max_disk_mb, disk_mb)
        
        # Luu thoi gian cua paper
        if paper_id and paper_time is not None:
            self.paper_times.append({
                'paper_id': paper_id,
                'time_seconds': paper_time
            })
        
    def finish(self, output_dir=None):
        """Ket thuc va in metrics"""
        self.end_time = time.time()
        total_time = self.end_time - self.start_time
        disk_increase = self.max_disk_mb - self.initial_disk_mb
        
        print("\n" + "=" * 60)
        print("KET QUA")
        print("=" * 60)
        
        # Thoi gian
        print("\nThoi gian:")
        print("   Tong: {:.2f}s ({:.2f} phut)".format(total_time, total_time/60))
        
        if self.paper_times:
            avg_time = sum(p['time_seconds'] for p in self.paper_times) / len(self.paper_times)
            print("   TB moi paper: {:.2f}s".format(avg_time))
            print("   So papers: {}".format(len(self.paper_times)))
        
        # Memory
        print("\nMemory:")
        print("   RAM max: {:.2f} MB ({:.2f} GB)".format(self.max_ram_mb, self.max_ram_mb/1024))
        current_ram = psutil.virtual_memory().used / (1024**2)
        print("   RAM hien tai: {:.2f} MB".format(current_ram))
        
        # Disk
        print("\nDisk:")
        print("   Disk max: {:.2f} MB ({:.2f} GB)".format(self.max_disk_mb, self.max_disk_mb/1024))
        print("   Tang: {:.2f} MB ({:.2f} GB)".format(disk_increase, disk_increase/1024))
        
        # Tinh kich thuoc output folder
        output_size_mb = 0
        if output_dir and os.path.exists(output_dir):
            total_size = sum(
                os.path.getsize(os.path.join(dp, f))
                for dp, dn, filenames in os.walk(output_dir)
                for f in filenames
            )
            output_size_mb = total_size / (1024**2)
            print("   Kich thuoc data: {:.2f} MB ({:.2f} GB)".format(output_size_mb, output_size_mb/1024))
        
        print("=" * 60)
        
        # Return dict de save
        return {
            'testbed': 'Google Colab CPU-only',
            'total_wall_time_seconds': total_time,
            'total_wall_time_minutes': total_time / 60,
            'total_wall_time_hours': total_time / 3600,
            'max_ram_mb': self.max_ram_mb,
            'max_ram_gb': self.max_ram_mb / 1024,
            'disk_increase_mb': disk_increase,
            'disk_increase_gb': disk_increase / 1024,
            'output_size_mb': output_size_mb,
            'output_size_gb': output_size_mb / 1024,
            'papers_processed': len(self.paper_times),
            'avg_time_per_paper': sum(p['time_seconds'] for p in self.paper_times) / len(self.paper_times) if self.paper_times else 0,
            'timestamp': datetime.now().isoformat()
        }

# Khoi tao monitor
monitor = PerformanceMonitor()
print("Monitor ready!")

Buoc 5: Chay scraper

Script se tu dong:
- Lay metadata tu arXiv API
- Download TeX sources (.tar.gz)
- Xoa hinh (png, jpg, pdf, eps)
- Lay references tu Semantic Scholar
- Luu theo cau truc de yeu cau

In [ ]:
# Script chay parallel scraper
%%writefile /content/ScrapingData/23127240/src/run_parallel.py
import os
import sys
import time
import logging

# Setup path
sys.path.insert(0, '/content/ScrapingData/23127240/src')

from config_settings import *
from utils import setup_logging, ensure_dir
from parallel_scraper import ParallelArxivScraper

# Setup logging
setup_logging(LOGS_DIR)
logger = logging.getLogger(__name__)

def main():
    logger.info("="*80)
    logger.info("BAT DAU CHAY SCRAPER")
    logger.info(f"MSSV: {STUDENT_ID}")
    logger.info(f"Pham vi: {START_YEAR_MONTH}.{START_ID:05d} den {END_YEAR_MONTH}.{END_ID:05d}")
    logger.info(f"So workers: {MAX_WORKERS}")
    logger.info("="*80)
    
    # Tao list cac paper IDs can cao
    paper_ids = []
    
    # Tinh toan: can bao nhieu papers tu thang dau
    TARGET_TOTAL = 5000
    total_in_last_month = END_ID
    papers_from_first_month = TARGET_TOTAL - total_in_last_month
    first_month_end_id = START_ID + papers_from_first_month - 1
    
    # Thang dau: tu START_ID den calculated end
    for paper_id in range(START_ID, first_month_end_id + 1):
        arxiv_id = f"{START_YEAR_MONTH}.{paper_id:05d}"
        paper_ids.append(arxiv_id)
    
    # Thang sau: tu 1 den END_ID
    for paper_id in range(1, END_ID + 1):
        arxiv_id = f"{END_YEAR_MONTH}.{paper_id:05d}"
        paper_ids.append(arxiv_id)
    
    logger.info(f"Tong so papers: {len(paper_ids)}")
    logger.info(f"Paper dau: {paper_ids[0]}")
    logger.info(f"Paper cuoi: {paper_ids[-1]}")
    
    # Setup thu muc output
    output_dir = DATA_DIR
    ensure_dir(output_dir)
    
    # Tao scraper
    scraper = ParallelArxivScraper(output_dir)
    
    # Check xem da co papers nao hoan thanh chua (de resume)
    completed = set()
    if os.path.exists(output_dir):
        for item in os.listdir(output_dir):
            item_path = os.path.join(output_dir, item)
            if os.path.isdir(item_path) and '-' in item:
                # Check xem paper nay da hoan thanh chua
                metadata_file = os.path.join(item_path, "metadata.json")
                references_file = os.path.join(item_path, "references.json")
                if os.path.exists(metadata_file) and os.path.exists(references_file):
                    arxiv_id = item.replace('-', '.')
                    completed.add(arxiv_id)
    
    if completed:
        logger.info(f"Da co {len(completed)} papers hoan thanh, bo qua chung")
        paper_ids = [pid for pid in paper_ids if pid not in completed]
        logger.info(f"Con lai: {len(paper_ids)} papers")
    
    # BAT DAU CAO!
    logger.info(f"\nBAT DAU cao voi {MAX_WORKERS} workers!")
    start_time = time.time()
    
    results = scraper.scrape_papers_batch(paper_ids, batch_size=50)
    
    elapsed = time.time() - start_time
    
    # In ket qua
    logger.info("\n" + "="*80)
    logger.info("HOAN THANH!")
    logger.info("="*80)
    logger.info(f"Thoi gian: {elapsed:.2f}s ({elapsed/60:.2f} phut)")
    logger.info(f"Thanh cong: {results['successful']}")
    logger.info(f"That bai: {results['failed']}")
    logger.info(f"Tong: {results['total']}")
    logger.info("="*80)

if __name__ == "__main__":
    main()

In [ ]:
import subprocess
import sys
import os
import time
import json

# Bat dau do wall time
monitor.start()

try:
    print("Dang chay scraper...")
    print("\nCac buoc (theo Lab 1):")
    print("  1. Entry Discovery - tim papers tren arXiv")
    print("  2. Download - tai source .tar.gz")
    print("  3. Xoa hinh - chi giu .tex va .bib")
    print("  4. References - cao tu Semantic Scholar")
    print("  5. Luu data - metadata.json, references.json")
    print("\nChay song song 6 workers!")
    print("=" * 70)
    
    # Chuyen vao thu muc src
    os.chdir('/content/ScrapingData/23127240/src')
    
    # Chay scraper voi realtime output
    process = subprocess.Popen(
        ['python3', '-u', 'run_parallel.py'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    # Stream output realtime
    return_code = None
    while True:
        line = process.stdout.readline()
        if not line:
            return_code = process.poll()
            if return_code is not None:
                break
            time.sleep(0.1)
            continue
        
        # In cac dong quan trong
        if "Progress:" in line or "Batch" in line or "HOAN THANH" in line:
            print("\n" + line.strip())
        elif "Scraping" in line or "Extracted" in line:
            print(".", end="", flush=True)
        else:
            print(line, end="", flush=True)
    
    # Doi process ket thuc
    process.wait()
    
    print("\n")
    if return_code != 0:
        print(f"Loi code: {return_code}")
    else:
        print("Scraper xong!")
    
    # Update metrics
    monitor.update_metrics()
    
    # Ve lai thu muc goc
    os.chdir('/content/ScrapingData/23127240')
    
except KeyboardInterrupt:
    print("\nDung boi user")
    if 'process' in locals():
        process.terminate()
    os.chdir('/content/ScrapingData/23127240')
except Exception as e:
    print(f"\nLoi: {e}")
    import traceback
    traceback.print_exc()
    os.chdir('/content/ScrapingData/23127240')
finally:
    # Ket thuc do wall time
    metrics = monitor.finish(output_dir="23127240_data")
    
    # Luu metrics
    with open('performance_metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    print("\nDa luu metrics: performance_metrics.json")

Buoc 6: Chay scraper

Sau khi chay xong, cac file se duoc tao tu dong:
- paper_details.csv - Chi tiet tung paper  
- scraping_stats.csv - Tong quan metrics
- scraping_stats.json - Du lieu day du

Luu y:
- KHONG tat Colab trong khi scraper dang chay
- Neu bi ngat, chay lai tu dau - code tu dong skip papers da scrape
- Progress duoc luu moi 50 papers

In [ ]:
# Chay script tao paper_details.csv
%cd /content/ScrapingData/23127240/src

print("Dang tao paper_details.csv...")
!python3 generate_paper_details_csv.py ../23127240_data

print("\nKiem tra file vua tao:")
!head -1 ../23127240_data/paper_details.csv
!echo "\nDong dau tien:"
!head -2 ../23127240_data/paper_details.csv | tail -1
!echo "\nTong so dong:"
!wc -l ../23127240_data/paper_details.csv

Buoc 7: Tao paper_details.csv voi day du 14 cot

Script nay se scan tat ca folders va tao CSV dung format:
paper_id,arxiv_id,title,authors,runtime_s,size_before,size_after,size_before_figures,size_after_figures,num_refs,current_output_size,max_rss,avg_rss,processed_at

Buoc 8: Download Du lieu

In [ ]:
import shutil
from google.colab import files

# Nen du lieu
print("Dang nen du lieu...")
shutil.make_archive('23127240_data', 'zip', '.', '23127240_data')
print(f"Da tao 23127240_data.zip")

# Kiem tra kich thuoc
size_mb = os.path.getsize('23127240_data.zip') / (1024**2)
print(f"Kich thuoc: {size_mb:.2f} MB")

if size_mb > 100:
    print("File lon hon 100MB, khuyen nghi upload len Google Drive")
    print("Chay cell tiep theo de upload len Drive")
else:
    print("\nBat dau download...")
    files.download('23127240_data.zip')

Buoc 9: Upload len Google Drive (neu file qua lon)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy vao Drive
!cp 23127240_data.zip /content/drive/MyDrive/
!cp performance_metrics.json /content/drive/MyDrive/

print("Da upload vao Google Drive:")
print("   - 23127240_data.zip")
print("   - performance_metrics.json")

---

GHI CHU

Yeu cau Lab 1 da hoan thanh:
- Testbed: Google Colab CPU-only mode
- Wall time measurement (end-to-end)
- Memory footprint (max RAM, disk usage)
- Scrape: TeX sources, metadata, references
- Remove figures de giam kich thuoc
- Cau truc theo format yeu cau

Rate Limiting:
- Semantic Scholar: 1 req/s, 100 req/5min
- Script co built-in retry mechanism

Demo Video (<=120s):
1. Setup (15s): Mo Colab, check CPU-only, clone repo
2. Running (45s): Chay scraper, show logs
3. Results (45s): Performance metrics, verify structure
4. Voice: Giai thich scraper design va reasoning

Lien he:
- Instructor: hlhdang@fit.hcmus.edu.vn

---

Chuc ban scraping thanh cong!